# Intro: CuPy and Numba on the GPU

10-20-2021


Useful links:
* [High Performance Python: CPUs](https://github.com/henryiii/python-performance-minicourse)
* [iscinumpy.gitlab.io](https://iscinumpy.gitlab.io)
* [CompClass](https://github.com/henryiii/compclass)

Note that we are using CPython 3.9. 3.10 is out, but is not quite ready for conda yet. And even when it is, Numba is slow to update due to heavy usage of bytecode, which is not (supposed to be) stable between releases.

## Problem 1: Negative Log Likelihood

Let's start with a NLL calculation. If you are doing an unbinned likelihood fit, this is main computation loop that drives that sort of fit. It's also _mostly_ embarrassingly parallel, except for a final reduction.

### NumPy (normal CPU solution)

Let's try with numpy so you can see the three lines of actual code involved. First our imports:

In [ ]:
import numpy as np
import math

Now we make some artificial data to run on (this is what we'd fit if we added the fitter):

In [ ]:
rng = np.random.default_rng(seed=42)

dist = np.hstack(
    [
        rng.normal(loc=1, scale=2.0, size=500_000),
        rng.normal(loc=1, scale=0.5, size=500_000),
    ]
)

Now we define a gaussian, product of two gaussians, and an nll function:

In [ ]:
def gaussian(x, μ, σ):
    return 1 / math.sqrt(2 * np.pi * σ**2) * np.exp(-((x - μ) ** 2) / (2 * σ**2))


def add(x, f_0, μ, σ_1, σ_2):
    return f_0 * gaussian(x, μ, σ_1) + (1 - f_0) * gaussian(x, μ, σ_2)


def nll(x, f_0, μ, σ_1, σ_2):
    return -np.sum(np.log(add(x, f_0, μ, σ_1, σ_2)))

Let's just show the actual value at the minimum for comparison later:

In [ ]:
nll(dist, 0.5, 1.0, 2.0, 0.5)

Let's see how much time this takes to compute:

In [ ]:
%%timeit
nll(
    dist,
    rng.random(),
    rng.normal(loc=1, scale=0.3),
    rng.normal(loc=2, scale=0.5),
    rng.normal(loc=0.5, scale=0.1),
)

FYI, this is _very_ good. NumPy is probably using multiple threads for parts of this computation, and fusing simple expressions.

### CuPy

#### CuPy drop-in

We are going to import cupy. `import cupy as cp` is very common, due to similarly with `np` (and you will sometimes see `import cupy as np`).

In [ ]:
import cupy

The first thing we need to do is move the NumPy array over to the GPU. We do that with `cupy.array`.

In [ ]:
cpdist = cupy.array(dist)

Actually, that's the last thing we need to do, as long as you have NumPy 1.18 or better. Everything works now:

In [ ]:
%%timeit
nll(
    cpdist,
    rng.random(),
    rng.normal(loc=1, scale=0.3),
    rng.normal(loc=2, scale=0.5),
    rng.normal(loc=0.5, scale=0.1),
).get()

NumPy 1.13 added the ability to override UFuncts, and 1.18 added the ability to override general functions, and CuPy uses this; you don't need to replace `np` with `cupy` unless you are making arrays (`array`, `asarray`, `empty`, `zeros`, etc.). If you do need to make an array, you can use `xp = cupy.get_array_module(existing_array)`, then `xp` will be either `numpy` or `cupy`, depending on the input array.

We can try to do better, though - cupy is making temporaries, which are costly. Since we are doing a reduction, let's write a reduction kernel:

In [ ]:
rku = cupy.ReductionKernel(
    "float64 x, float64 f_0, float64 mean, float64 sigma, float64 sigma2",
    "float64 r",
    """
    log(     f_0  * rsqrt(2*M_PI*sigma*sigma)   * exp(-(x-mean)*(x-mean)/(2*sigma*sigma)) +
        (1 - f_0) * rsqrt(2*M_PI*sigma2*sigma2) * exp(-(x-mean)*(x-mean)/(2*sigma2*sigma2)))
    """,
    "a + b",
    "r = -a",
    "0",
    "redu_kernel",
)

In [ ]:
def nll(dist, f_0, mean, sigma, sigma2):
    return rku(dist, f_0, mean, sigma, sigma2)

In [ ]:
nll(cpdist, 0.5, 1.0, 2.0, 0.5).get()

In [ ]:
%%timeit
nll(
    cpdist,
    rng.random(),
    rng.normal(loc=1, scale=0.3),
    rng.normal(loc=2, scale=0.5),
    rng.normal(loc=0.5, scale=0.1),
).get()

This is actually a bit worse. We did much better in the middle, not needing as many temporaries, but did much worse in the reduction, as this is not as optimized as `cp.sum`. Let's try a hybrid solution:

In [ ]:
mykernel = cupy.ElementwiseKernel(
    "float64 x, float64 f_0, float64 mean, float64 sigma, float64 sigma2",
    "float64 z",
    """
    
    double s12 = 2*sigma*sigma;
    double s22 = 2*sigma2*sigma2;
    
    double p = -(x-mean)*(x-mean);
    double g = rsqrt(M_PI*s12) * exp(p/s12);
    double g2 = rsqrt(M_PI*s22) * exp(p/s22);
    
    z = log(f_0 * g + (1 - f_0) * g2);
        
    """,
    "mykernel",
)

In [ ]:
def nll(dist, f_0, mean, sigma, sigma2):
    return -cupy.sum(mykernel(dist, f_0, mean, sigma, sigma2))

In [ ]:
nll(cpdist, 0.5, 1.0, 2.0, 0.5).get()

In [ ]:
%%timeit
nll(
    cpdist,
    rng.random(),
    rng.normal(loc=1, scale=0.3),
    rng.normal(loc=2, scale=0.5),
    rng.normal(loc=0.5, scale=0.1),
).get()

This is optimal - we are using the CUB sum as well as avoiding temporaries.

## Numba GPU

Another solution is Numba's JIT compiler, which supports CUDA.

In [ ]:
import numba.cuda
import math


@numba.cuda.jit("float64(float64,float64,float64)", device=True, inline=True)
def gaussian(x, μ, σ):
    return 1 / math.sqrt(2 * np.pi * σ**2) * math.exp(-((x - μ) ** 2) / (2 * σ**2))


@numba.vectorize(["float64(float64,float64,float64,float64,float64)"], target="cuda")
def log_add(x, f_0, mean, sigma, sigma2):
    return -math.log(
        f_0 * gaussian(x, mean, sigma) + (1 - f_0) * gaussian(x, mean, sigma2)
    )


@numba.cuda.reduce
def sum_reduce(a, b):
    return a + b


def nll(dist, f_0, mean, sigma, sigma2):
    return sum_reduce(log_add(dist, f_0, mean, sigma, sigma2))

Numba and CuPy support 0-cost transfer between libraries, so you can select the tool that's best for you! We'll make a Numba device vector from our CuPy one. `cupy.asarray(nbdist)` would convert back.

In [ ]:
nbdist = numba.cuda.to_device(cpdist)

In [ ]:
nll(nbdist, 0.5, 1.0, 2.0, 0.5)

In [ ]:
%%timeit
nll(
    nbdist,
    rng.random(),
    rng.normal(loc=1, scale=0.3),
    rng.normal(loc=2, scale=0.5),
    rng.normal(loc=0.5, scale=0.1),
)

This is basically on par with the ReductionKernel, as expected.